<a href="https://colab.research.google.com/github/Maucho6340/Telecom_X_LATAM_Predict/blob/main/Challenge_predict_evasion_op1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'> CHALLENGE TELECOM X:
**Análisis de evasión de clientes Parte-2**

## **MISIÓN**



Tu nueva misión es desarrollar modelos predictivos capaces de prever qué clientes tienen mayor probabilidad de cancelar sus servicios.

La empresa quiere anticiparse al problema de la cancelación, y te corresponde a ti construir un pipeline robusto para esta etapa inicial de modelado.

## **OBJETIVOS DEL DESAFÍO**



- Preparar los datos para el modelado (tratamiento, codificación, normalización).

- Realizar análisis de correlación y selección de variables.

- Entrenar dos o más modelos de clasificación.

- Evaluar el rendimiento de los modelos con métricas.

- Interpretar los resultados, incluyendo la importancia de las variables.

- Crear una conclusión estratégica señalando los principales factores que influyen en la cancelación.

## **PLAN DE TRABAJO**


A.- **Preparar los datos para el modelado**.<br>

1.	Crear el repositorio de tu proyecto en GitHub.<br>

2.	Extracción del Archivo Tratado.<br><br>

B.- **Limpieza**.<br>

3.	Eliminación de Columnas Irrelevantes.<br>
3-1.	Elimina columnas que no aportan valor al análisis o a los modelos predictivos, como identificadores únicos (por ejemplo, el ID del cliente).<br> Estas columnas no ayudan en la predicción de la cancelación y pueden incluso perjudicar el desempeño de los modelos.
<br>


C.- **Encoding**.<br>

4.	Verificación de la Proporción de Cancelación (Churn).<br>
4-1.	Calcula la proporción de clientes que cancelaron en relación con los que permanecieron activos. Evalúa si existe un desbalance entre las clases, ya que esto puede impactar en los modelos predictivos y en el análisis de los resultados.<br>
Puedes usar value_counts() de pandas para obtener esta proporción.<br>

5.	Balanceo de Clases (opcional)<br>
5-1.	Si deseas profundizar en el análisis, aplica técnicas de balanceo como undersampling o oversampling. En situaciones de fuerte desbalanceo, herramientas como SMOTE pueden ser útiles para generar ejemplos sintéticos de la clase minoritaria.<br>

6.	Normalización o Estandarización (si es necesario)<br>
6-1.	Evalúa la necesidad de normalizar o estandarizar los datos, según los modelos que se aplicarán. Modelos basados en distancia, como KNN, SVM, Regresión Logística y Redes Neuronales, requieren este preprocesamiento. Por otro lado, modelos basados en árboles, como Decision Tree, Random Forest y XGBoost, no son sensibles a la escala de los datos.
<br>


D.- **Correlación y Selección de Variables**.<br>

7.	Análisis de Correlación<br>
7-1.	Visualiza la matriz de correlación para identificar relaciones entre las variables numéricas. Presta especial atención a las variables que muestran una mayor correlación con la cancelación, ya que estas pueden ser fuertes candidatas para el modelo predictivo.<br>

8.	Análisis Dirigido<br>
8-1.	Investiga cómo variables específicas se relacionan con la cancelación, tales como:<br>
A)	Tiempo de contrato × Cancelación<br>
B)	Gasto total × Cancelación<br>
8-2.	Utiliza gráficos como boxplots o scatter plots para visualizar patrones y posibles tendencias.
<br>


E.- **Modelado Predictivo**.<br>

9.	Separación de Datos<br>
9-1.	Divide el conjunto de datos en entrenamiento y prueba para evaluar el rendimiento del modelo. Una división común es 70% para entrenamiento y 30% para prueba, o 80/20, dependiendo del tamaño de la base de datos.<br>

10.	Creación de Modelos<br>
10-.1.	Crea al menos dos modelos diferentes para predecir la cancelación de clientes.<br>
A)	Un modelo puede requerir normalización, como Regresión Logística o KNN.<br>
B)	El otro modelo puede no requerir normalización, como Árbol de Decisión o Random Forest.<br>
10-2.	La decisión de aplicar o no la normalización depende de los modelos seleccionados. Ambos modelos pueden ser creados sin normalización, pero también es una opción combinar modelos con y sin normalización.<br>
A)	Justificación:<br>
a)	Regresión Logística / KNN: Estos modelos son sensibles a la escala de los datos, por lo que la normalización es importante para que los coeficientes o las distancias se calculen correctamente.<br>
b)	Árbol de Decisión / Random Forest: Estos modelos no dependen de la escala de los datos, por lo que no es necesario aplicar normalización.<br>
c)	Si decides normalizar los datos, deberías explicar cómo esta etapa asegura que los modelos basados en distancia o en optimización de parámetros no se vean sesgados por la magnitud de las variables.<br>

11.	Evaluación de los Modelos<br>
11-1.	Evalúa cada modelo utilizando las siguientes métricas:<br>
A)	Exactitud (Acurácia)
B)	Precisión
C)	Recall
D)	F1-score
E)	Matriz de confusión<br>
11-2.	Después, realiza un análisis crítico y compara los modelos:<br>
A)	¿Cuál modelo tuvo el mejor desempeño?<br>
B)	¿Algún modelo presentó overfitting o underfitting? Si es así, considera las posibles causas y ajustes:<br>
a)	Overfitting: Cuando el modelo aprende demasiado sobre los datos de entrenamiento, perdiendo la capacidad de generalizar a nuevos datos. Considera reducir la complejidad del modelo o aumentar los datos de entrenamiento.
b)	Underfitting: Cuando el modelo no captura bien las tendencias de los datos, lo que indica que es demasiado simple. Intenta aumentar la complejidad del modelo o ajustar sus parámetros.<br>


F.- **Interpretación y Conclusiones**.
<br>

12.	Análisis de la Importancia de las Variables<br>
12-1.	Después de elegir los modelos, realiza el análisis de las variables más relevantes para la predicción de la cancelación:<br>
A)	Regresión Logística: Investiga los coeficientes de las variables, que muestran su contribución a la predicción de cancelación.<br>
B)	KNN (K-Nearest Neighbors): Observa cómo los vecinos más cercanos influyen en la decisión de clasificación. Las variables más impactantes pueden ser aquellas que más contribuyen a la proximidad entre los puntos de datos.<br>
C)	Random Forest: Utiliza la importancia de las variables proporcionada por el modelo. Random Forest calcula la importancia basándose en cómo cada variable contribuye a la reducción de la impureza durante las divisiones de los árboles.<br>
D)	SVM (Support Vector Machine): En el SVM, las variables más relevantes son aquellas que influyen en la frontera de decisión entre las clases. Puedes analizar los coeficientes de los vectores de soporte para entender qué variables tienen mayor impacto.<br>
E)	Otros Modelos: Dependiendo del modelo elegido, considera el análisis de métricas específicas para comprender la relevancia de las variables. Por ejemplo, coeficientes en modelos lineales, pesos en redes neuronales, o la importancia relativa en técnicas de boosting (como XGBoost).<br>

13.	Conclusión<br>
13-1.	Elaboren un informe detallado, destacando los factores que más influyen en la cancelación, basándose en las variables seleccionadas y en el rendimiento de cada modelo.<br>
13-2.	Identifiquen los principales factores que afectan la cancelación de clientes y propongan estrategias de retención basadas en los resultados obtenidos.


## **A.- PREPARAR LOS DATOS PARA EL MODELADO**

### A.1 <u>Crear repositorio en Git Hub</u>

In [2]:
# Repositorio GitHub: Maucho6340/Telecom_X_LATAM_predict

### A.2 <u>Extracción del Archivo Tratado</u>

In [3]:
# Código A.2 – Extracción del Archivo Tratado
import pandas as pd

# Ruta del archivo
ruta_archivo = "/content/df_predict_churn_2.csv"

# Cargar el archivo en un DataFrame
df = pd.read_csv(ruta_archivo)

# Verificar la forma y las primeras filas
print("Shape del DataFrame:", df.shape)
df.head()

Shape del DataFrame: (7267, 24)


,ID_Cliente,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,Facturacion_Mensual,account.Charges.Total,Cuentas_Diarias,Churn_bin,Churn_num
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.186667,0.0,0.0
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,1.996667,0.0,0.0
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fibra Óptica,...,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.463333,1.0,1.0
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fibra Óptica,...,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.266667,1.0,1.0
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fibra Óptica,...,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.796667,1.0,1.0


## **B.- LIMPIEZA**

### B.3 <u>Eliminar columnas irrelevantes</u>

In [4]:
# Eliminación de columnas irrelevantes
df.drop(columns=['ID_Cliente'], inplace=True)

# Verificación del cambio
print("Columnas restantes:")
print(df.columns.tolist())

Columnas restantes:
['Churn', 'customer.gender', 'customer.SeniorCitizen', 'customer.Partner', 'customer.Dependents', 'customer.tenure', 'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService', 'internet.OnlineSecurity', 'internet.OnlineBackup', 'internet.DeviceProtection', 'internet.TechSupport', 'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract', 'account.PaperlessBilling', 'account.PaymentMethod', 'Facturacion_Mensual', 'account.Charges.Total', 'Cuentas_Diarias', 'Churn_bin', 'Churn_num']


## **C.- ENCODING**

### C.4.1 <u>Verificación de la Proporción de Cancelación (Churn)</u>

In [5]:
# Código C.4.1 – Verificación de la Proporción de Cancelación (Churn)
proporcion_churn = df['Churn'].value_counts(normalize=True)

# Mostrar proporciones
print("Proporción de cancelación (Churn):")
print(proporcion_churn)

Proporción de cancelación (Churn):
Churn
No             0.711986
Yes            0.257190
Desconocido    0.030824
Name: proportion, dtype: float64


### C.4.2 <u>Eliminación de registros con Churn desconocido</u>

In [6]:
# Código C.4.3 – Eliminación de registros con Churn desconocido
df = df[df['Churn'] != 'Desconocido'].copy()

# Verificamos que se haya eliminado correctamente
print("Distribución actual de Churn:")
print(df['Churn'].value_counts(normalize=True))

Distribución actual de Churn:
Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64


### C.5.1 <u>Balanceo de Clases</u>

#### Codificación de variables categóricas

In [7]:
# C.6.1 – Codificación de variables categóricas
df_encoded = pd.get_dummies(df, drop_first=True)

# C.6.2 – Verificación de codificación
print("Shape del DataFrame codificado:", df_encoded.shape)
print("Tipos de datos después de codificación:")
print(df_encoded.dtypes.value_counts())

Shape del DataFrame codificado: (7043, 34)
Tipos de datos después de codificación:
bool       27
float64     5
int64       2
Name: count, dtype: int64


#### Separación de variables

In [8]:
# C.5.2 – Separación de variables (versión final)
X = df_encoded.drop(columns=['Churn_Yes', 'Churn_bin', 'Churn_num'])  # Variables predictoras
y = df_encoded['Churn_num']  # Variable objetivo binaria


#### Imputación de valores faltantes

In [9]:
# C.5.2.1 – Imputación de valores faltantes
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # Imputación con la media
X_imputed = imputer.fit_transform(X)      # Resultado: array NumPy

# Reconstrucción del DataFrame original
import pandas as pd
X = pd.DataFrame(X_imputed, columns=X.columns)

#### Aplicación de SMOTE

In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Distribución después de SMOTE:")
print(pd.Series(y_resampled).value_counts(normalize=True))


Distribución después de SMOTE:
Churn_num
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64


### C 6.1	<u>Normalización o Estandarización</u>

In [19]:
from sklearn.preprocessing import StandardScaler

# 1. Identificar columnas numéricas
numeric_cols = X_resampled.select_dtypes(include=['float64', 'int64']).columns

# 2. Inicializar el escalador
scaler = StandardScaler()

# 3. Aplicar escalado solo a columnas numéricas
X_resampled_scaled = X_resampled.copy()
X_resampled_scaled[numeric_cols] = scaler.fit_transform(X_resampled[numeric_cols])

# 4. Confirmar forma y tipo
print("Shape después de escalar:", X_resampled_scaled.shape)
print("Tipos de datos:\n", X_resampled_scaled.dtypes)

Shape después de escalar: (10348, 31)
Tipos de datos:
 customer.SeniorCitizen                           float64
customer.tenure                                  float64
Facturacion_Mensual                              float64
account.Charges.Total                            float64
Cuentas_Diarias                                  float64
customer.gender_Male                             float64
customer.Partner_Yes                             float64
customer.Dependents_Yes                          float64
phone.PhoneService_Yes                           float64
phone.MultipleLines_No phone service             float64
phone.MultipleLines_Yes                          float64
internet.InternetService_Fibra Óptica            float64
internet.InternetService_Sin Servicio            float64
internet.OnlineSecurity_No internet service      float64
internet.OnlineSecurity_Yes                      float64
internet.OnlineBackup_No internet service        float64
internet.OnlineBackup_Yes        

### C.5.2 <u>Identificación de variables categóricas</u>

In [11]:
# C.5.2 – Separación de variables predictoras y objetivo
X = df.drop(columns=['Churn', 'Churn_bin', 'Churn_num'])  # Variables predictoras
y = df['Churn_num']  # Variable objetivo binaria (0 = No, 1 = Yes)

### C.5.3 <u>Aplicación de SMOTE</u>

In [12]:
# C.5.3 – Aplicación de SMOTE
from imblearn.over_sampling import SMOTE
import pandas as pd

# Instanciamos SMOTE
smote = SMOTE(random_state=42)

# Aplicamos SMOTE
X_resampled, y_resampled = smote.fit_resample(X, y)

# Verificamos el nuevo balance
print("Distribución después de SMOTE:")
print(pd.Series(y_resampled).value_counts(normalize=True))

ValueError: could not convert string to float: 'Female'

Observación
1. Hay un desbalance moderado entre "No" y "Yes", lo que podría afectar métricas como recall y F1-score.
1. La categoría "Desconocido" representa un 3.1% del total. Esto puede indicar:<br>

- Valores faltantes que fueron etiquetados como "Desconocido".
- Un error en la codificación original del dataset.

Recomendación<br>
Antes de continuar, sería ideal:
1. Revisar el origen de "Desconocido": ¿proviene de valores nulos o de una categoría real?
2. Decidir cómo tratarla:
- Eliminar esos registros si son pocos y no aportan valor.
- Imputar o reclasificar si tienen sentido contextual.
- Excluir "Desconocido" del target si se va a entrenar un modelo binario.


### **Filtrado de registros válidos**

In [ ]:
# Filtrar solo registros con Churn conocido
df_valid = df[df['Churn'].isin(['Yes', 'No'])].copy()

# Verificación
print(df_valid['Churn'].value_counts(normalize=True))

- Eliminamos registros ambiguos o mal etiquetados en la variable Churn.
- Calculamos proporciones para entender el grado de desbalance.
- Resultado: distribución clara entre "Yes" y "No" que justifica aplicar técnicas de balanceo como SMOTE.


### Codificación

In [ ]:
# Aplicar codificación one-hot a las variables categóricas
df_encoded = pd.get_dummies(df_valid, drop_first=True)

# Verificación del resultado
print(f"Shape después del encoding: {df_encoded.shape}")
print("Primeras columnas codificadas:")
print(df_encoded.columns[:10].tolist())

#### **Separación de Variables Predictoras (X) y Variable Objetivo (y)**

In [13]:
# Definir la variable objetivo
y = df_encoded['Churn_bin']

# Definir el conjunto de variables predictoras, excluyendo columnas redundantes
X = df_encoded.drop(columns=['Churn_bin', 'Churn_num', 'Churn_Yes'])

# Verificación
print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")

Shape de X: (7043, 31)
Shape de y: (7043,)


#### **Revisión de Correlaciones Perfectas**


In [14]:
# Calcular la matriz de correlación
corr_matrix = X.corr()

# Identificar pares de variables con correlación perfecta (1 o -1)
perfect_corrs = []

for col in corr_matrix.columns:
    for row in corr_matrix.index:
        if col != row and abs(corr_matrix.loc[row, col]) == 1.0:
            pair = tuple(sorted([col, row]))
            if pair not in perfect_corrs:
                perfect_corrs.append(pair)

# Mostrar resultados
print("Pares con correlación perfecta:")
for pair in perfect_corrs:
    print(pair)

Pares con correlación perfecta:
('phone.MultipleLines_No phone service', 'phone.PhoneService_Yes')
('internet.InternetService_Sin Servicio', 'internet.OnlineSecurity_No internet service')
('internet.InternetService_Sin Servicio', 'internet.OnlineBackup_No internet service')
('internet.DeviceProtection_No internet service', 'internet.InternetService_Sin Servicio')
('internet.InternetService_Sin Servicio', 'internet.TechSupport_No internet service')
('internet.InternetService_Sin Servicio', 'internet.StreamingTV_No internet service')
('internet.InternetService_Sin Servicio', 'internet.StreamingMovies_No internet service')
('internet.OnlineBackup_No internet service', 'internet.OnlineSecurity_No internet service')
('internet.DeviceProtection_No internet service', 'internet.OnlineSecurity_No internet service')
('internet.OnlineSecurity_No internet service', 'internet.TechSupport_No internet service')
('internet.OnlineSecurity_No internet service', 'internet.StreamingTV_No internet service'

#### **Eliminación de Variables Redundantes**

In [15]:
# Eliminar columnas redundantes
cols_to_drop = [
    'phone.MultipleLines_No phone service',
    'internet.OnlineSecurity_No internet service',
    'internet.OnlineBackup_No internet service',
    'internet.DeviceProtection_No internet service',
    'internet.TechSupport_No internet service',
    'internet.StreamingTV_No internet service',
    'internet.StreamingMovies_No internet service'
]

X_clean = X.drop(columns=cols_to_drop)

# Verificación
print(f"Shape de X después de limpieza: {X_clean.shape}")

Shape de X después de limpieza: (7043, 24)


### Limpieza previa a SMOTE

### División y balanceo

En este caso, como el objetivo es predecir si un cliente va a cancelar o no el servicio, estamos trabajando con una variable categórica (churn: sí o no). Por eso, los modelos adecuados son los de clasificación, como DecisionTreeClassifier o KNeighborsClassifier

### <font color='blue'>**1.- Preparar los datos para el modelado (tratamiento, codificación, normalización)**.

##### **Tratamiento de valores nulos**

In [16]:
# Imputar con mediana en 'account.Charges.Total' — versión segura
df_final['account.Charges.Total'] = df_final['account.Charges.Total'].fillna(df_final['account.Charges.Total'].median())

# Eliminar filas sin datos en la variable objetivo
df_final.dropna(subset=['Chrun_bin', 'Chrun_num'], inplace=True)

NameError: name 'df_final' is not defined

##### **Codificación de Variables Categórica**

In [ ]:
# Elimina columna ID_Cliente o l ignora si ya no existe
df_final.drop(columns=['ID_Cliente'], inplace=True, errors='ignore')

# Codificación con pd.get_dummies
columnas_categoricas = df_final.select_dtypes(include='object').columns

df_final = pd.get_dummies(df_final, columns=columnas_categoricas, drop_first=True)

##### **Verificación final del DataFram**

In [17]:
# Confirmar que no quedan datos nulos ni columnas tipo object
print(df_final.isnull().sum())
print(df_final.dtypes)

NameError: name 'df_final' is not defined

#### **Resumen 1a Etapa**

Esta etapa se centró en garantizar la calidad, consistencia y estructura adecuada del dataset para su uso en modelos de machine learning.

##### Acciones realizadas:

- **Carga del archivo**: Se verificó la estructura del DataFrame, compuesto por 7,267 registros y 24 columnas.
- **Tratamiento de valores nulos**: Se imputaron o eliminaron valores faltantes, especialmente en `account.Charges.Total`, `Churn_bin` y `Churn_num`.
- **Codificación de variables categóricas**: Se aplicó One-Hot Encoding a variables como género, tipo de contrato, servicios de internet, método de pago, etc.
- **Verificación final del DataFrame**:
  - Todas las variables relevantes quedaron sin valores nulos.
  - Las variables booleanas fueron correctamente codificadas como `bool`.
  - Las variables numéricas (`customer.tenure`, `Facturacion_Mensual`, `account.Charges.Total`, `Cuentas_Diarias`) quedaron listas para análisis estadístico.



**Conclusión de la Etapa**<br>

Se completó exitosamente la etapa de exploración y limpieza del conjunto de datos, abordando valores faltantes, inconsistencias y codificación de variables categóricas. Este proceso permitió establecer una base estructurada y confiable para el análisis posterior. Las decisiones tomadas se documentaron cuidadosamente, priorizando la reproducibilidad y la trazabilidad del flujo de trabajo. El dataset resultante presenta una estructura coherente, libre de anomalías críticas, y está listo para avanzar hacia la selección de variables relevantes.

### <font color='blue'>**2.- Realizar análisis de correlación y selección de variables**.

##### **Análisis de correlación numérica**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Correlación de variables numéricas
correlaciones = df_final.corr(numeric_only=True)

# Visualizar correlación con la variable objetivo
plt.figure(figsize=(10, 6))
sns.heatmap(correlaciones[['Chrun_num']].sort_values(by='Chrun_num', ascending=False),
            annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlación de variables numéricas con Churn_num")
plt.tight_layout()
plt.show()

##### **Análisis de Importancia para Variables Booleana**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
X = df_final.drop(columns=['Chrun_bin', 'Chrun_num', 'Chrun_Yes'])  # excluimos el target y su duplicado
y = df_final['Chrun_bin']  # o 'Churn_num' según tu criterio

modelo_arbol = DecisionTreeClassifier(random_state=42)
modelo_arbol.fit(X, y)

importancias = pd.Series(modelo_arbol.feature_importances_, index=X.columns)
importancias_ordenadas = importancias.sort_values(ascending=False)

# Visualizamos nuevamente
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(x=importancias_ordenadas[:15], y=importancias_ordenadas.index[:15], palette='viridis')
plt.title("Top 15 variables más importantes (sin Chrun_Yes)")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.tight_layout()
plt.show()

Variables Predictivas Seleccionadas para el Modelo **DecisionTreeClassifier**

| Variable                                  | Descripción y Justificación |
|--------------------------------------------|------------------------------|
| `customer.tenure`                          | Antigüedad del cliente; menor permanencia se asocia a mayor churn. |
| `account.Charges.Total`                    | Total facturado; refleja exposición financiera acumulada. |
| `internet.InternetService_Fibra Óptica`    | Este tipo de servicio está correlacionado con mayor abandono. |
| `Cuentas_Diarias`                          | Actividad operativa diaria; puede influir en la permanencia. |
| `Facturacion_Mensual`                      | Costo mensual del servicio; niveles altos pueden generar abandono. |
| `customer.SeniorCitizen`                   | Grupo demográfico con patrones de churn distintos. |
| `account.PaymentMethod_Electronic check`   | Método de pago vinculado a mayor volatilidad de permanencia. |
| `account.Contract_Two year`                | Contrato largo suele estar ligado a menor churn. |
| `internet.OnlineSecurity_Yes`              | Servicios adicionales pueden aumentar la percepción de valor. |
| `phone.MultipleLines_Yes`                  | Presencia de líneas múltiples podría asociarse a mayor complejidad o costos. |

##### **Análisis de Multicolinealidad con VIF**

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

# Usamos df_final directamente con las variables seleccionadas
X = df_final[['customer.tenure', 'account.Charges.Total', 'Cuentas_Diarias', 'Facturacion_Mensual']]
X_const = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i+1) for i in range(len(X.columns))]

print(vif_data)

NameError: name 'df_final' is not defined

##### **Visualización de correlaciones cruzadas**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(X.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlación entre Variables Predictivas")
plt.show()

##### **Comparación con otras técnicas de selección automática**

<u>Recursive Feature Elimination (RFE)</u>

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
rfe = RFE(model, n_features_to_select=5)
rfe.fit(X, y)

selected_features = X.columns[rfe.support_]
print("Variables seleccionadas por RFE:", selected_features.tolist())

<u>SelectFromModel</u>

In [ ]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(model)
sfm.fit(X, y)

selected_features_sfm = X.columns[sfm.get_support()]
print("Variables seleccionadas por SelectFromModel:", selected_features_sfm.tolist())

##### **Resumen Etapa 2**

Esta etapa tuvo como objetivo identificar las variables más relevantes para la predicción de chrun, reducir la dimensionalidad y evitar redundancias.

### Análisis de correlación numérica

Se exploraron relaciones entre variables continuas. Se detectó una correlación casi perfecta entre `Facturacion_Mensual` y `Cuentas_Diarias`, lo que sugiere redundancia informativa.

### Análisis de importancia para variables booleanas

Se entrenó un `DecisionTreeClassifier` para obtener la importancia relativa de las variables categóricas codificadas. Se visualizaron las 15 variables más influyentes en la predicción de chrun.

### Análisis de multicolinealidad con VIF

Se calculó el **Factor de Inflación de Varianza (VIF)** para las variables numéricas seleccionadas:

| Variable                | VIF        | Interpretación                        |
|------------------------|------------|---------------------------------------|
| `customer.tenure`      | 5.77       | Aceptable                             |
| `account.Charges.Total`| 9.40       | Alta multicolinealidad                |
| `Cuentas_Diarias`      | ∞          | Multicolinealidad extrema             |
| `Facturacion_Mensual`  | ∞          | Multicolinealidad extrema             |

Este análisis confirmó la necesidad de revisar la inclusión simultánea de Facturacion_Mensual y Cuentas_Diarias en modelos sensibles a colinealidad.
Los valores de VIF tienden a infinito debido a la correlación perfecta entre ambas variables (coeficiente de correlación = 1.00).


### 🔹 Visualización de correlaciones cruzadas

Se generaron heatmaps para variables categóricas codificadas y variables numéricas, lo que permitió validar visualmente las relaciones detectadas por VIF y el árbol de decisión.

### 🔹 Comparación con técnicas de selección automática

Se aplicaron dos métodos complementarios:

- **RFE (Recursive Feature Elimination)**: Seleccionó las siguientes variables:
  ```python
  ['customer.tenure', 'account.Charges.Total', 'Cuentas_Diarias', 'Facturacion_Mensual']
- **'SelectFromModel'**: Seleccionó la siguiente variable:
   ['account.Charges.Total']

**Conclusión de la Etapa**<br>

Se consolidó un conjunto de cuatro variables altamente relevantes para el modelado, seleccionadas mediante técnicas automáticas y análisis interpretativo. Aunque se detectó multicolinealidad entre dos de ellas, se decidió mantenerlas temporalmente para evaluar su impacto en el rendimiento de los modelos en la siguiente etapa. La convergencia entre métodos como RFE, SelectFromModel y el análisis de importancia del árbol de decisión aporta solidez y confianza al proceso de preparación


### <font color='blue'>**3.- Entrenar dos o más modelos de clasificación**.

In [ ]:
# 📦 Importar librerías
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# 🎯 División inicial del dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ⚖️ Rebalanceo con SMOTE (solo sobre el conjunto de entrenamiento)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 📏 Escalado de variables
scaler = StandardScaler()
X_train_res_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)  # Solo transformamos, no fit

# 🔁 Validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 🧪 Definición de modelos
modelo_lr = LogisticRegression(solver='liblinear', class_weight='balanced', max_iter=1000, random_state=42)
modelo_rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# 📊 Validación cruzada con F1-score
scores_lr = cross_val_score(modelo_lr, X_train_res_scaled, y_train_res, cv=cv, scoring='f1')
scores_rf = cross_val_score(modelo_rf, X_train_res_scaled, y_train_res, cv=cv, scoring='f1')

# 📈 Resultados
print("F1-score promedio (Logistic Regression):", scores_lr.mean())
print("F1-score promedio (Random Forest):", scores_rf.mean())

### <font color='blue'>**4.- Evaluación del redimiento de los modelos**.

##### valuación sobre X_test_scaled y y_test
Importar métrica


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

##### Entrenar modelos finales sobre todo el conjunto rebalanceado

In [ ]:
# Entrenar sobre todo el conjunto rebalanceado y escalado
modelo_lr.fit(X_train_res_scaled, y_train_res)
modelo_rf.fit(X_train_res_scaled, y_train_res)

##### Predicciones sobre el conjunto de prueba

In [ ]:
y_pred_lr = modelo_lr.predict(X_test_scaled)
y_pred_rf = modelo_rf.predict(X_test_scaled)

##### Métricas para cada molo

In [ ]:
def evaluar_modelo(y_true, y_pred, nombre):
    print(f"\nEvaluación para {nombre}")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))
    print("Matriz de confusión:\n", confusion_matrix(y_true, y_pred))
    print("Reporte de clasificación:\n", classification_report(y_true, y_pred))

# Evaluar ambos modelos
evaluar_modelo(y_test, y_pred_lr, "Logistic Regression")
evaluar_modelo(y_test, y_pred_rf, "Random Forest")

**Interpretación técnica**<br>
Ambos modelos tienen rendimiento similar, con una ligera ventaja para Logistic Regression en todas las métricas.<br>
La clase minoritaria (churn) tiene métricas moderadas:
+ Precision ≈ 0.57–0.59 → el modelo acierta en ~57% de los casos que predice como churn.
+ Recall ≈ 0.58–0.62 → el modelo detecta ~60% de los churn reales.<br>

La matriz de confusión muestra que hay falsos negativos y falsos positivos, pero no extremos.


- Los modelos no están sobreajustados: el rendimiento en test es coherente con la validación cruzada.
- El rebalanceo con SMOTE ayudó a mejorar la sensibilidad hacia la clase minoritaria.
- Aún hay espacio para mejorar recall sin sacrificar demasiado la precisión
- - -

### <font color='blue'>**5.- Interpretar los resultados, incluyendo la importancia de las variables**.

##### Importancia de variables en Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

# Entrenar modelo
logreg_model_final = LogisticRegression(max_iter=1000, random_state=42)
logreg_model_final.fit(X_train_resampled, y_train_resampled)

# Extraer coeficientes
coefficients = logreg_model_final.coef_[0]
feature_names = X_train_resampled.columns

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
}).sort_values(by='Abs_Coefficient', ascending=False)

# Mostrar top 10
coef_df.head(10)

##### Importancia de variables en Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Entrenar modelo
rf_model_final = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_final.fit(X_train_resampled, y_train_resampled)

# Extraer importancia
rf_importances = rf_model_final.feature_importances_
feature_names = X_train_resampled.columns

rf_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_importances
}).sort_values(by='Importance', ascending=False)

# Mostrar top 10
rf_df.head(10)

- Hay consistencia con Logistic Regression: tenure, Charges.Total, Contract_Two year, Facturacion_Mensual, InternetService_Fibra Óptica aparecen en ambos modelos como relevantes.
- Random Forest no indica dirección (positivo/negativo), pero confirma la importancia estructural de estas variables.


##### Comparativa top 10: Logistic Regression vs Random Forest

| Variable                                 | LR: Coeficiente | RF: Importancia | Coinciden | Interpretación clave                          |
|------------------------------------------|------------------|------------------|-----------|------------------------------------------------|
| customer.tenure                          | -1.58            | 0.152            | ✅        | Más antigüedad, menos churn                    |
| account.Charges.Total                    | +0.87            | 0.128            | ✅        | Cargos altos → más churn                       |
| internet.InternetService_Fibra Óptica    | +0.81            | 0.041            | ✅        | Fibra óptica → más churn                       |
| account.Contract_Two year                | -0.63            | 0.063            | ✅        | Contrato largo → retención                     |
| Facturacion_Mensual                      | -0.59            | 0.095            | ✅        | Menor facturación → menos churn                |
| Cuentas_Diarias                          | -0.59            | 0.097            | ✅        | Más actividad → menos churn                    |
| account.Contract_One year                | -0.29            | 0.032            | ✅        | Contrato medio → retención                     |
| internet.StreamingTV_Yes                 | +0.29            | —                | ❌        | Streaming TV → más churn                       |
| internet.StreamingMovies_Yes             | +0.26            | —                | ❌        | Streaming Movies → más churn                   |
| phone.MultipleLines_Yes                  | +0.23            | —                | ❌        | Múltiples líneas → más churn                   |

Se evaluaron dos modelos predictivos de churn: Regresión Logística y Random Forest, con el objetivo de identificar las variables más influyentes en la decisión de abandono del cliente.
Ambos modelos coincidieron en destacar:
- Antigüedad del cliente (tenure)
- Cargos acumulados (Charges.Total)
- Tipo de contrato (Contract_Two year, Contract_One year)
- Facturación mensual
- Actividad diaria<br>

Estas variables deben considerarse prioritarias en estrategias de retención.<br>

La regresión logística aporta dirección (efecto positivo o negativo), mientras que Random Forest confirma la relevancia estructural de las variables. La consistencia entre modelos refuerza la confiabilidad de los hallazgos.


### <font color='blue'>**6.- Crear una conclusión estratégica señalando los principales factores que influyen en la cancelación**.

##### **a) <u>Conclusión estratégica sobre el churn**</u>

El análisis de modelos predictivos revela un conjunto claro de factores que influyen significativamente en la cancelación del servicio. Estos hallazgos permiten orientar acciones concretas para reducir el churn y mejorar la retención.



##### **b) <u>Principales factores asociados al churn**</u>

- **Baja antigüedad del cliente (`tenure`)**  
  Clientes nuevos presentan mayor riesgo de cancelación. Los primeros meses son críticos para establecer valor y fidelidad.

- **Altos cargos acumulados (`Charges.Total`)**  
  La percepción de costos elevados está fuertemente asociada al abandono. Es clave revisar políticas de facturación y comunicar beneficios.

- **Tipo de contrato**  
  Contratos de mayor duración (1 o 2 años) se asocian con menor churn. Incentivar compromisos a largo plazo puede ser efectivo.

- **Facturación mensual**  
  Menor facturación mensual se vincula con menor churn, posiblemente por percepción de accesibilidad o valor.

- **Nivel de actividad (`Cuentas_Diarias`)**  
  Clientes más activos tienden a permanecer. Fomentar el uso frecuente del servicio fortalece la relación.

- **Tipo de servicio contratado**  
  Servicios como fibra óptica y streaming se asocian con mayor churn. Es importante revisar la experiencia de usuario en estos segmentos.


##### **c) <u>Recomendación estratégica</u>**

Las acciones de retención deben centrarse en:

- Fortalecer la experiencia de los clientes nuevos.
- Ofrecer incentivos por contratos largos.
- Revisar la estructura de precios y cargos acumulados.
- Promover el uso activo del servicio.
- Monitorear segmentos con servicios premium para detectar fricciones.